<a href="https://colab.research.google.com/github/sieun-Bae/deep-learning/blob/master/text_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
# Import regularizers
from tensorflow.keras import regularizers
import tensorflow.keras.utils as ku 
import numpy as np 

### 1. Use hard-coded text to provide training data


In [32]:
tokenizer = Tokenizer()

data = "The series begins with the return of Upper East Side teenage it girl \
        Serena van der Woodsen (Blake Lively) from a mysterious absence. \
        She is reunited with her frenemy Blair Waldorf (Leighton Meester) and \
        her mother Lily van der Woodsen (Kelly Rutherford), while she also meets \
        Dan Humphrey (Penn Badgley), an aspiring writer from Brooklyn. \
        Other main characters are played by Chace Crawford, Ed Westwick, Taylor \
        Momsen, Jessica Szohr, Matthew Settle and Kaylee DeFer."

corpus = data.lower().split("\n")

tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

input_sequences = []
for line in corpus:
  token_list = tokenizer.texts_to_sequences([line])[0]
  for i in range(1, len(token_list)):
    n_gram_sequence = token_list[:i+1]
    input_sequences.append(n_gram_sequence)

max_sequence_len = max([len(x) for x in input_sequences])

input_sequences = np.array(pad_sequences(input_sequences, maxlen = max_sequence_len, padding='pre'))

xs = input_sequences[:, :-1]
labels = input_sequences[:, -1]

ys = tf.keras.utils.to_categorical(labels, num_classes = total_words)

model = Sequential()
model.add(Embedding(total_words, 64, input_length = max_sequence_len -1))
model.add(Bidirectional(LSTM(20)))
model.add(Dense(total_words, activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.fit(xs, ys, epochs = 500, verbose = 1)

Train on 73 samples
Epoch 1/500
73/73 [==============================] - 1s 16ms/sample - loss: 4.1913 - acc: 0.0274
Epoch 2/500
73/73 [==============================] - 1s 7ms/sample - loss: 4.1858 - acc: 0.0411
Epoch 3/500
73/73 [==============================] - 0s 7ms/sample - loss: 4.1815 - acc: 0.0411
Epoch 4/500
73/73 [==============================] - 1s 8ms/sample - loss: 4.1774 - acc: 0.0685
Epoch 5/500
73/73 [==============================] - 1s 7ms/sample - loss: 4.1737 - acc: 0.0685
Epoch 6/500
73/73 [==============================] - 1s 7ms/sample - loss: 4.1694 - acc: 0.0685
Epoch 7/500
73/73 [==============================] - 1s 7ms/sample - loss: 4.1651 - acc: 0.0959
Epoch 8/500
73/73 [==============================] - 1s 7ms/sample - loss: 4.1602 - acc: 0.1096
Epoch 9/500
73/73 [==============================] - 0s 7ms/sample - loss: 4.1551 - acc: 0.1096
Epoch 10/500
73/73 [==============================] - 1s 8ms/sample - loss: 4.1495 - acc: 0.1096
Epoch 11/500
73/73

### 2. Test the  text generator model

In [24]:
seed_text = "To be or not to be"
next_words = 10

for _ in range(next_words):
  token_list = tokenizer.texts_to_sequences([seed_text])[0]
  token_list = pad_sequences([token_list], maxlen = max_sequence_len-1, padding='pre')
  predicted = model.predict_classes(token_list, verbose=0)
  output_word = ""
  for word, index in tokenizer.word_index.items():
    if index == predicted:
      output_word = word
      break
  seed_text += " " + output_word

print(seed_text)


To be or not to be series series begins with the return of upper east side


### 3. Use a Larger Dataset

Below is the code for Shakespeare's sonnets

In [25]:
tokenizer = Tokenizer()
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sonnets.txt \
    -O /tmp/sonnets.txt
data = open('/tmp/sonnets.txt').read()

corpus = data.lower().split("\n")


tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

# create input sequences using list of tokens
input_sequences = []
for line in corpus:
	token_list = tokenizer.texts_to_sequences([line])[0]
	for i in range(1, len(token_list)):
		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)


# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]

label = ku.to_categorical(label, num_classes=total_words)

--2019-11-27 06:37:44--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sonnets.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.119.128, 2a00:1450:4013:c00::80
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.119.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93578 (91K) [text/plain]
Saving to: ‘/tmp/sonnets.txt’

/tmp/sonnets.txt    100%[===================>]  91.38K  --.-KB/s    in 0.001s  

2019-11-27 06:37:44 (156 MB/s) - ‘/tmp/sonnets.txt’ saved [93578/93578]



### 4. Define neural network using LSTM

In [43]:
tokenizer = Tokenizer()

data = "The series begins with the return of Upper East Side teenage it girl \
        Serena van der Woodsen (Blake Lively) from a mysterious absence. \
        She is reunited with her frenemy Blair Waldorf (Leighton Meester) and \
        her mother Lily van der Woodsen (Kelly Rutherford), while she also meets \
        Dan Humphrey (Penn Badgley), an aspiring writer from Brooklyn. \
        Other main characters are played by Chace Crawford, Ed Westwick, Taylor \
        Momsen, Jessica Szohr, Matthew Settle and Kaylee DeFer."

corpus = data.lower().split("\n")

tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

input_sequences = []
for line in corpus:
  token_list = tokenizer.texts_to_sequences([line])[0]
  for i in range(1, len(token_list)):
    n_gram_sequence = token_list[:i+1]
    input_sequences.append(n_gram_sequence)

max_sequence_len = max([len(x) for x in input_sequences])

model = Sequential()
model.add(Embedding(total_words, 100, input_length = max_sequence_len-1)) # Try 100 dimensions
model.add(LSTM(150)) # Try 150 units 
model.add(Dropout(0.2)) # Try dropping out 20% of neurons
model.add(LSTM(100)) # Try 100 units
# A Dense Layer including regularizers – not covered in lectures but useful to know
model.add(Dense(total_words/2, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(total_words, activation='softmax')) # Output layer
adam = Adma(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(xs, ys, epochs=100, verbose=1)
print(model.summary())


ValueError: ignored

### 5. Train the model

In [0]:
 history = model.fit(predictors, label, epochs=50, verbose=1)

### 6. Plot the accuracy and loss

In [0]:
import matplotlib.pyplot as plt
acc = history.history['acc']
loss = history.history['loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training accuracy')
plt.title('Training accuracy')

plt.figure()

plt.plot(epochs, loss, 'b', label='Training Loss')
plt.title('Training loss')
plt.legend()

plt.show()

### 7. Test the poetry generator!

In [0]:
seed_text = "Help me Obi Wan Kenobi, you're my only hope"
next_words = 100
  
for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
	predicted = model.predict_classes(token_list, verbose=0)
	output_word = ""
	for word, index in tokenizer.word_index.items():
		if index == predicted:
			output_word = word
			break
	seed_text += " " + output_word
print(seed_text)